# End-to-End Change Detection using a Pretrained Model and a Public Dataset

This notebook demonstrates how to use the `change-detection` codebase to perform an end-to-end change detection task using a small, publicly available dataset. We will:

- Download and prepare the LEVIR-CD dataset (a commonly used dataset for building change detection).
- Preprocess the data.
- Initialize the model using the provided codebase.
- Train the model.
- Evaluate the model.
- Visualize some results.

In [ ]:
# Install necessary libraries (if not already installed)
# !pip install torch torchvision matplotlib tqdm
# !pip install albumentations

In [ ]:
# Import necessary libraries
import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm

# Add the path to the change-detection codebase
sys.path.append('/path/to/change-detection')  # Update this path accordingly

# Import modules from the codebase
from data.datasets import ChangeDetectionDataset, get_default_transforms
from models.siamese_unet import SiameseUNet
from train.loss_functions import DiceLoss
from train.metrics import IoU, F1Score
from train.trainer import CustomTrainer, TrainerConfig


## 1. Download and Prepare the Dataset

We will use the **LEVIR-CD** dataset, which contains high-resolution images for building change detection tasks. For the sake of this demo, we'll use a small subset of the dataset.

**Note**: Ensure you have the necessary permissions to download and use the dataset.

In [ ]:
# Create directories for data
os.makedirs('data/LEVER-CD/train/images_before', exist_ok=True)
os.makedirs('data/LEVER-CD/train/images_after', exist_ok=True)
os.makedirs('data/LEVER-CD/train/labels', exist_ok=True)
os.makedirs('data/LEVER-CD/val/images_before', exist_ok=True)
os.makedirs('data/LEVER-CD/val/images_after', exist_ok=True)
os.makedirs('data/LEVER-CD/val/labels', exist_ok=True)


In [ ]:
# For demonstration, let's assume we've downloaded a few image pairs and labels
# Normally, you would download and extract the dataset here
# For this demo, we'll create some dummy data

from PIL import Image, ImageDraw

def create_dummy_image(path):
    img = Image.new('RGB', (256, 256), color='white')
    draw = ImageDraw.Draw(img)
    draw.rectangle([50, 50, 200, 200], outline='black', fill='gray')
    img.save(path)

def create_dummy_label(path):
    img = Image.new('L', (256, 256), color=0)
    draw = ImageDraw.Draw(img)
    draw.rectangle([50, 50, 200, 200], fill=255)
    img.save(path)

# Create dummy training data
for i in range(5):
    create_dummy_image(f'data/LEVER-CD/train/images_before/image_{i}_before.png')
    create_dummy_image(f'data/LEVER-CD/train/images_after/image_{i}_after.png')
    create_dummy_label(f'data/LEVER-CD/train/labels/label_{i}.png')

# Create dummy validation data
for i in range(2):
    create_dummy_image(f'data/LEVER-CD/val/images_before/image_{i}_before.png')
    create_dummy_image(f'data/LEVER-CD/val/images_after/image_{i}_after.png')
    create_dummy_label(f'data/LEVER-CD/val/labels/label_{i}.png')


## 2. Prepare Data Loaders

We will use the `ChangeDetectionDataset` class from the codebase to create PyTorch datasets and data loaders.


In [ ]:
# Define paths
train_before_paths = [f'data/LEVER-CD/train/images_before/image_{i}_before.png' for i in range(5)]
train_after_paths = [f'data/LEVER-CD/train/images_after/image_{i}_after.png' for i in range(5)]
train_label_paths = [f'data/LEVER-CD/train/labels/label_{i}.png' for i in range(5)]

val_before_paths = [f'data/LEVER-CD/val/images_before/image_{i}_before.png' for i in range(2)]
val_after_paths = [f'data/LEVER-CD/val/images_after/image_{i}_after.png' for i in range(2)]
val_label_paths = [f'data/LEVER-CD/val/labels/label_{i}.png' for i in range(2)]


In [ ]:
# Create datasets
train_dataset = ChangeDetectionDataset(
    image_pairs=list(zip(train_before_paths, train_after_paths)),
    labels=train_label_paths,
    transform=get_default_transforms()
)

val_dataset = ChangeDetectionDataset(
    image_pairs=list(zip(val_before_paths, val_after_paths)),
    labels=val_label_paths,
    transform=get_default_transforms()
)


In [ ]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)


## 3. Initialize the Model

We will use the `SiameseUNet` model from the codebase.


In [ ]:
# Initialize the model
model = SiameseUNet(in_channels=3, out_channels=1)


## 4. Define Loss Function and Metrics


In [ ]:
# Define loss function and metrics
loss_fn = DiceLoss()

metrics = [
    IoU(threshold=0.5),
    F1Score(threshold=0.5)
]


## 5. Set Up the Trainer


In [ ]:
# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Define trainer configuration
trainer_config = TrainerConfig(
    num_epochs=5,
    checkpoint_path=None,
    checkpoint_dir='checkpoints/',
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

# Initialize the trainer
trainer = CustomTrainer(
    model=model,
    optimizer=optimizer,
    loss_fn=loss_fn,
    metrics=metrics,
    config=trainer_config
)


## 6. Train the Model


In [ ]:
# Train the model
trainer.train(train_loader, val_loader)


## 7. Evaluate the Model

After training, we can evaluate the model on the validation set.


In [ ]:
# Set model to evaluation mode
model.eval()

# Initialize metrics
for metric in metrics:
    metric.reset()

# Run evaluation
with torch.no_grad():
    for inputs, targets in val_loader:
        inputs = [input_tensor.to(trainer.device) for input_tensor in inputs]
        targets = targets.to(trainer.device)
        outputs = model(*inputs)
        loss = loss_fn(outputs, targets)
        for metric in metrics:
            metric.update(outputs, targets)

# Compute and print metrics
for metric in metrics:
    metric_value = metric.compute()
    print(f'{metric.__class__.__name__}: {metric_value:.4f}')


## 8. Visualize Some Results

Let's visualize some predictions from the validation set.


In [ ]:
# Function to display images
def show_images(images, titles=None):
    n = len(images)
    plt.figure(figsize=(15, 5))
    for i in range(n):
        plt.subplot(1, n, i+1)
        img = images[i]
        if img.ndim == 2:
            plt.imshow(img, cmap='gray')
        else:
            plt.imshow(img)
        if titles:
            plt.title(titles[i])
        plt.axis('off')
    plt.show()


In [ ]:
# Get a batch from validation loader
inputs, targets = next(iter(val_loader))
inputs = [input_tensor.to(trainer.device) for input_tensor in inputs]
targets = targets.to(trainer.device)

# Get model predictions
model.eval()
with torch.no_grad():
    outputs = model(*inputs)

# Move tensors to CPU and convert to numpy arrays
before_image = inputs[0].cpu().numpy()[0].transpose(1, 2, 0)
after_image = inputs[1].cpu().numpy()[0].transpose(1, 2, 0)
target = targets.cpu().numpy()[0, 0]
prediction = torch.sigmoid(outputs).cpu().numpy()[0, 0] > 0.5

# Display images
show_images(
    [before_image, after_image, target, prediction],
    titles=['Before Image', 'After Image', 'Ground Truth', 'Prediction']
)


## Conclusion

We have demonstrated an end-to-end change detection workflow using the provided codebase and a small dataset. This includes data preparation, model training, evaluation, and visualization of results.

For a production environment, you would use the full dataset and potentially more advanced models and training configurations.
